# Exploratory Data Analysis (EDA) on House Prices Dataset

## Introduction

In this project, we will perform **Exploratory Data Analysis (EDA)** on the **House Prices: Advanced Regression Techniques** dataset from Kaggle. The goal of EDA is to understand the dataset by examining its structure, cleaning the data, and visualizing key features. This process will help us uncover important patterns and relationships in the data, and provide insights into which features might be useful for predictive modeling in future steps.

We will follow these 7 key steps throughout the analysis:

### Steps of the EDA Process

1. **Understanding the Data**: This includes looking at the data types of various columns, understanding what each column represents, and identifying any key columns.
2. **Summary Statistics**: Calculating basic statistics like mean, median, mode, standard deviation, and range for numerical columns.
3. **Data Cleaning**: Identifying and handling missing values, outliers, and any inconsistencies in the data.
4. **Data Visualization**: Creating visualizations such as histograms, box plots, scatter plots, and correlation matrices to get a sense of the distributions, relationships, and patterns in the data.
5. **Feature Engineering**: Creating new features or modifying existing ones to better capture the underlying patterns in the data.
6. **Identifying Trends and Patterns**: Using the summary statistics and visualizations to identify any obvious trends, patterns, or anomalies in the data.
7. **Initial Hypothesis Testing**: Formulating and testing initial hypotheses about the data based on the observations from the EDA.

---

Let's begin by loading the dataset and reviewing its structure.

In [9]:
import pandas as pd
df = pd.read_csv('../data/train.csv')

## Understanding the Data ##
For the first step of Exploratory Data Analysis (EDA), we'll begin by familiarizing ourselves with the structure of the dataset and the information provided. This involves inspecting the first few rows to understand the general structure (columns and data types).
in this case we also have a very helpful accompanying documentation (data_description.txt) to understand the meaning of each column.
Identifying key columns that may have important roles in the analysis, based on the data description.

In [6]:


# Step 3: Display the first few rows of the dataset to get a general overview
print("First 5 rows of the dataset:")
print(df.head())

# Step 4: Output the summary of the dataset (columns and data types)
print("\nDataset Information:")
print(df.info())

# Step 5: Output summary statistics for numerical columns
print("\nSummary statistics for numerical columns:")
print(df.describe())


First 5 rows of the dataset:
   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCondition  Sale

## Data Cleaning

Data cleaning is an essential step in preparing our dataset for analysis. In this section, we will handle missing values, deal with inconsistencies, and prepare the data for further analysis.

### 1. Handle Missing Values

First, we need to check which columns contain missing values and how we can handle them. Missing values can be dropped, imputed with mean/median/mode, or treated in other ways depending on the data type and context.


In [10]:
# Checking for missing values in the dataset
missing_data = df.isnull().sum().sort_values(ascending=False)
missing_percentage = (df.isnull().sum() / df.isnull().count() * 100).sort_values(ascending=False)
missing_data_table = pd.concat([missing_data, missing_percentage], axis=1, keys=['Missing Values', '% of Total Values'])

# Display the columns with missing values
print("Columns with missing values and their percentages:")
print(missing_data_table[missing_data_table['Missing Values'] > 0])

# Handle missing values:
# - Drop columns where more than 50% of the data is missing
# - For numerical columns, we can impute missing values with the median.
# - For categorical columns, we can impute missing values with the mode.

# Drop columns where more than 50% of data is missing
threshold = 0.5 * len(df)
removed_columns = df.columns[df.isnull().sum() > threshold].tolist()

# Drop columns where more than 50% of data is missing
df = df.drop(columns=removed_columns)

# Print the list of removed columns
if removed_columns:
    print(f"Removed columns (more than 50% missing data): {removed_columns}")
else:
    print("No columns were removed due to missing data.")

# Replace missing values in numerical columns with the median and print the details
for col in df.select_dtypes(include=['float64', 'int64']):
    if df[col].isnull().sum() > 0:
        median_value = df[col].median()
        df[col] = df[col].fillna(median_value)
        print(f"Missing values in column '{col}' replaced by the median: {median_value}")

# Replace missing values in categorical columns with the mode and print the details
for col in df.select_dtypes(include=['object']):
    if df[col].isnull().sum() > 0:
        mode_value = df[col].mode()[0]
        df[col] = df[col].fillna(mode_value)
        print(f"Missing values in column '{col}' replaced by the mode: '{mode_value}'")


Columns with missing values and their percentages:
              Missing Values  % of Total Values
PoolQC                  1453          99.520548
MiscFeature             1406          96.301370
Alley                   1369          93.767123
Fence                   1179          80.753425
MasVnrType               872          59.726027
FireplaceQu              690          47.260274
LotFrontage              259          17.739726
GarageYrBlt               81           5.547945
GarageCond                81           5.547945
GarageType                81           5.547945
GarageFinish              81           5.547945
GarageQual                81           5.547945
BsmtFinType2              38           2.602740
BsmtExposure              38           2.602740
BsmtQual                  37           2.534247
BsmtCond                  37           2.534247
BsmtFinType1              37           2.534247
MasVnrArea                 8           0.547945
Electrical                 1         

### 2. Handle Inconsistent Data Types

Often, datasets have columns with inconsistent or incorrect data types (e.g., numerical data stored as strings). We will ensure that each column has the correct data type.

??convert to 'category'

In [15]:
# Convert MSSubClass to a categorical variable as it represents a class, not a numerical value
#df['MSSubClass'] = df['MSSubClass'].astype('category')

# Find and display examples of mixed data types within a column
for col in df.columns:
    mixed_types = df[col].apply(type).nunique() > 1  # Check if there are more than one data type
    if mixed_types:
        # Get the unique data types in the column
        unique_types = df[col].apply(type).unique()
        
        # Print column name and the different types found
        print(f"Column '{col}' contains mixed data types: {unique_types}")
        
        # Display examples of each data type
        print(f"Examples of different types in column '{col}':")
        for data_type in unique_types:
            examples = df[col][df[col].apply(lambda x: isinstance(x, data_type))].head(3).tolist()
            print(f" - {data_type}: {examples}")
        print()  # Blank line for readability

todo: correct types based on data_description.txt

### 3. Handle Outliers

Outliers can distort the analysis, especially in regression tasks. We will detect and handle outliers using the IQR (Interquartile Range) method for continuous variables.

In [ ]:
# Function to remove outliers using IQR (Interquartile Range)
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # Remove rows with outliers
    df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Apply the function to key continuous columns
# For example, removing outliers from 'GrLivArea' and 'LotArea'
df = remove_outliers(df, 'GrLivArea')
df = remove_outliers(df, 'LotArea')


### 4. Standardize Categorical Variables

Categorical variables often need to be standardized or encoded for analysis. In this step, we will handle categorical variables by encoding them into numerical representations.


In [ ]:
# 4. Standardizing Categorical Variables

# Use one-hot encoding for categorical variables
df = pd.get_dummies(df, drop_first=True)

# Display the updated dataframe
print("Updated dataframe after one-hot encoding:")
print(df.head())


### 5. Final Check

After performing the cleaning steps, we'll do a final check to ensure there are no remaining missing values or inconsistencies.


In [ ]:
# 5. Final Check

# Recheck for any remaining missing values
final_missing_data = df.isnull().sum().sum()
if final_missing_data == 0:
    print("Data cleaning complete. No missing values remain.")
else:
    print(f"Warning: There are still {final_missing_data} missing values in the dataset.")

# Display the final cleaned dataset
print("Final cleaned dataset overview:")
print(df.info())
